<a href="https://colab.research.google.com/github/alexriley7/luigi_datapipeline-for-Ubuntu/blob/main/luigi_datapipeline_for_Ubuntu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

--2022-09-02 22:19:07--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299321244 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop3.tgz’

spark-3.3.0-bin-had 100%[===================>] 285.45M   244MB/s    in 1.2s    

2022-09-02 22:19:09 (244 MB/s) - ‘spark-3.3.0-bin-hadoop3.tgz’ saved [299321244/299321244]



In [2]:
import os

In [3]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import findspark

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [8]:
mkdir luigi-demo

In [9]:
cd luigi-demo

/content/luigi-demo


In [12]:
pip install wheel luigi beautifulsoup4 requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 13.1 MB/s 
  Created wheel for luigi: filename=luigi-3.1.1-py3-none-any.whl size=1082710 sha256=d5eb9c9b4205313f19868450b9d65e5d464ffd64faf4bc13ad0adc02bf1bbbfb
  Stored in directory: /root/.cache/pip/wheels/44/2d/25/2996d02abce93a0ee0ce98f206c8a7241f1ac89c5150f3430f
Successfully built luigi


In [15]:
import luigi

class HelloLuigi(luigi.Task):

    def output(self):
        return luigi.LocalTarget('hello-luigi.txt')

    def run(self):
        with self.output().open("w") as outfile:
            outfile.write("Hello Luigi!")



In [17]:
python3 -m luigi --module hello-world HelloLuigi --local-scheduler

SyntaxError: ignored

In [ ]:
mkdir data
mkdir data/counts
mkdir data/downloads

In [ ]:
import requests
import luigi
from bs4 import BeautifulSoup


class GetTopBooks(luigi.Task):
    """
    Get list of the most popular books from Project Gutenberg
    """

    def output(self):
        return luigi.LocalTarget("data/books_list.txt")

    def run(self):
        resp = requests.get("http://www.gutenberg.org/browse/scores/top")

        soup = BeautifulSoup(resp.content, "html.parser")

        pageHeader = soup.find_all("h2", string="Top 100 EBooks yesterday")[0]
        listTop = pageHeader.find_next_sibling("ol")

        with self.output().open("w") as f:
            for result in listTop.select("li>a"):
                if "/ebooks/" in result["href"]:
                    f.write("http://www.gutenberg.org{link}.txt.utf-8\n"
                        .format(
                            link=result["href"]
                        )
                    )

In [ ]:
python -m luigi --module word-frequency GetTopBooks --local-scheduler

In [ ]:
cat data/books_list.txt

In [ ]:
sudo ufw allow 8082/tcp

In [ ]:
sudo sh -c ". luigi-venv/bin/activate ;luigid --background --port 8082"

In [ ]:
luigid --port 8082 > /dev/null 2> /dev/null &

In [ ]:
python -m luigi --module word-frequency GetTopBooks

In [ ]:
. . .
class DownloadBooks(luigi.Task):
    """
    Download a specified list of books
    """
    FileID = luigi.IntParameter()

    REPLACE_LIST = """.,"';_[]:*-"""

    def requires(self):
        return GetTopBooks()

    def output(self):
        return luigi.LocalTarget("data/downloads/{}.txt".format(self.FileID))

    def run(self):
        with self.input().open("r") as i:
            URL = i.read().splitlines()[self.FileID]

            with self.output().open("w") as outfile:
                book_downloads = requests.get(URL)
                book_text = book_downloads.text

                for char in self.REPLACE_LIST:
                    book_text = book_text.replace(char, " ")

                book_text = book_text.lower()
                outfile.write(book_text)

In [ ]:
class CountWords(luigi.Task):
    """
    Count the frequency of the most common words from a file
    """

    FileID = luigi.IntParameter()

    def requires(self):
        return DownloadBooks(FileID=self.FileID)

    def output(self):
        return luigi.LocalTarget(
            "data/counts/count_{}.pickle".format(self.FileID),
            format=luigi.format.Nop
        )

    def run(self):
        with self.input().open("r") as i:
            word_count = Counter(i.read().split())

            with self.output().open("w") as outfile:
                pickle.dump(word_count, outfile)

In [ ]:
class TopWords(luigi.Task):
    """
    Aggregate the count results from the different files
    """

    def requires(self):
        requiredInputs = []
        for i in range(GlobalParams().NumberBooks):
            requiredInputs.append(CountWords(FileID=i))
        return requiredInputs

    def output(self):
        return luigi.LocalTarget("data/summary.txt")

    def run(self):
        total_count = Counter()
        for input in self.input():
            with input.open("rb") as infile:
                nextCounter = pickle.load(infile)
                total_count += nextCounter

        with self.output().open("w") as f:
            for item in total_count.most_common(GlobalParams().NumberTopWords):
                f.write("{0: <15}{1}\n".format(*item))